# Preprocessing
 
Dans ce notebook de preprocessing, je transforme mon dataset d'images et de xmls en deux sous datasets de train et de test, avec chacun son fichier json au format coco pour utiliser lors de l'entrainement du modele. 
Il faut au préalable avoir un dossier avec toutes les images, et un dossier avec tous les xmls.


* Conversion du Dataset de xmls à json (COCO format)
* Séparation en deux dossiers de train et test, aléatoirement ou selon des indices spécifiques

## Imports et Dossiers



In [ ]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random
from shutil import copyfile, rmtree, copytree
pd.set_option('display.max_colwidth', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Data est le dosssier mère contenant les sous dossiers d'images et d'xmls
Data = "drive/MyDrive/Mines Nancy/Depinfo/Projet/Faster_rCNN/Data_detectron"

# Dossiers des images et xmls
image_path = Data + "/images"
xml_path = Data + "/xmls"

# On va séparer les images en train et test, dans les sous dossiers train_path et test_path
train_path = Data + "/train"
test_path = Data + "/test"

# Pour la représentation des box
COLORS = ['b', 'r', 'm', 'y', 'w', 'k']

# Il n'y a qu'une seule classe
category_id = {"smoke" : "0"}

#  On vérifie qu'il y a autant d'images que de xmls et on extrait les noms dans des listes
images_name = os.listdir(image_path)
xmls_name = os.listdir(xml_path)

xmls_name.sort()
images_name.sort()

print("Il y a", len(images_name), "images, et", len(xmls_name), "annotations xmls.")


## Retirer certaines images du jeu de donnée si nécessaire

In [ ]:
# Si on souhaite retirer des images en faisant notre train/test ( Par exemple les images faites artificiellement )
# Il faut changer l'intervalle 0:150 en fonction des images que l'on souhaite garder

xmls_name = xmls_name[0:150]
images_name = images_name[0:150]
assert len(xmls_name) == len(images_name), "Il n'y a pas autant d'images que d'annotations"

print("Il y a", len(images_name), "images, et", len(xmls_name), "annotations xmls.")

## Fonctions pour explorer le jeu de donnée

In [ ]:
# Prend en argument un fichier xml décrivant une image
# Renvoie une liste contenant le nom du fichier de l'image correspondant, sa hauteur, sa largeur, et une liste de toutes les bbox de fumées sur cette image
# chaque bbox est une liste des coordonnées [x0, y0, x1, y1, nom_de_la_classe]

def getData(xml_name):
    tree = ET.parse(xml_path + '/' + xml_name)
    annotation = tree.getroot()

    filepath = annotation.find("filename")
    filename = filepath.text
    
    size = annotation.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    objects = annotation.findall("object")
    bbox = []
    for object in objects:
        class_name = object.find("name").text
        class_name = class_name.lower()
        coord = object.find("bndbox")

        xmin = int(float(coord.find("xmin").text))
        xmax = int(float(coord.find("xmax").text))
        ymin = int(float(coord.find("ymin").text))
        ymax = int(float(coord.find("ymax").text))

        bbox.append([xmin, ymin, xmax, ymax, class_name])

    return [filename, height, width, bbox]

# Transforme le résultat de getData en un DataFrame Panda
def xml_to_df(data):
  res=pd.DataFrame(columns=["file_name","height","width","annotations"])
  res["annotations"]=res["annotations"].astype('object')

  l = []
  for bbox in data[3]:
    new_bbox = [bbox[0], bbox[1], bbox[2]-bbox[0], bbox[3] - bbox[1]]
    l.append({"bbox" : new_bbox, "bbox_mode" : 1, "category_id" : category_id.get(bbox[4])})

  res.at[0,"file_name"] = data[0]
  res.at[0,"annotations"] = l
  res.at[0,"height"] = data[1]
  res.at[0,"width"] = data[2]
  return res


# Affiche l'image et dessine les box associée au data
# Data de format renvoyé par getData()
def plot_data(data):
    im = Image.open(os.path.join(image_path, data[0]))
    fig, ax = plt.subplots()
    ax.imshow(im)
    for box in data[3]:
        xmin, ymin, xmax, ymax, class_name = box
        w, h = xmax - xmin, ymax - ymin

        color = random.choice(COLORS)
        rect = patches.Rectangle((xmin, ymin), w, h, fill=False, color=color)

        ax.add_patch(rect)
        # Que de la fumée donc pas trop nécessaire
        plt.text(xmin, ymin - 5, class_name, color=color)
    plt.show()

In [ ]:
# Exemple
data_test = getData(xmls_name[0])
plot_data(data_test)

## Fonctions pour créer le jeu de donnée

In [ ]:
# Sépare les images en train et test
# Met chaque image dans un dossier test et train
# renvoie les DataFrame pandas associés et crée les .json
# Si randomize = True, le choix se fait aléatoirement
# Sinon, c'est dans l'ordre alphabétique
# pourcentTrain représente le pourcentage d'image que l'on veut en train
def make_train_test(randomize = False, pourcentTrain = 0.7):
    
    assert (len(xmls_name) == len(images_name))
    
    n = len(xmls_name)
    nTrain = int(n * pourcentTrain)
    
    ## Si on veut aléatoire ou non
    if randomize : 
        xmls_name_alea = random.sample(xmls_name, n)
    else:
        xmls_name_alea = xmls_name


    print("Générer le train dataset", )
    main_result_train = train_panda(0, nTrain, xmls_name_alea)
    
    print("Générer le test dataset")
    main_result_test = test_panda(nTrain, n, xmls_name_alea)

    return main_result_train, main_result_test

# Ces fonctions verifie que le dossier de train et de test est bien vide, et le vident sinon
def reinitialize_test():
    if os.path.exists(test_path):
        rmtree(test_path)
    
    os.mkdir(test_path)

def reinitialize_train():
    if os.path.exists(train_path):
        rmtree(train_path)
    
    os.mkdir(train_path)

# Genere le fichier json à partir du dataframe Panda renvoyé
def generate_jsons(main_result, name):
    main_result.reset_index(inplace=True)
    main_result.rename(columns={"index":"image_id"},inplace=True)
    main_result.to_json(name,orient="records")

# Ces fonctions crée les dossiers de test et de train, séparement
# Les images de numero entre i_deb et i_fin seront prises.
# xmls est le fichier contenant tous les noms de xmls ( xmls_name normalement )

def test_panda(i_deb, i_fin, xmls = xmls_name):
    
    reinitialize_test()
    main_result_test=pd.DataFrame(columns=["file_name","height","width","annotations"])
    main_result_test["annotations"]=main_result_test["annotations"].astype('object')
    
    for i in range(i_deb, i_fin):
        
        data = getData(xmls[i])
        
        # Copy the image to the train directory
        filepath = os.path.join(test_path, data[0])
        copyfile(os.path.join(image_path, data[0]), filepath)
        
        res = xml_to_df(data)
        main_result_test = main_result_test.append(res)
        main_result_test.reset_index(drop=True,inplace=True)
        print(round(100*(i - i_deb)/(i_fin - i_deb), 2), "%")
    generate_jsons(main_result_test, test_path + "/dataset.json")
    return main_result_test

def train_panda(i_deb, i_fin, xmls):    
    
    reinitialize_train()
    main_result_train=pd.DataFrame(columns=["file_name","height","width","annotations"])
    main_result_train["annotations"]=main_result_train["annotations"].astype('object')
    
    for i in range(i_deb, i_fin):
        
        data = getData(xmls[i])
        
        # Copy the image to the train directory
        filepath = os.path.join(train_path, data[0])
        copyfile(os.path.join(image_path, data[0]), filepath)
        
        res = xml_to_df(data)
        main_result_train = main_result_train.append(res)
        main_result_train.reset_index(drop=True,inplace=True)
        print(round(100*(i - i_deb)/(i_fin - i_deb), 2), "%")
    generate_jsons(main_result_train, train_path + "/dataset.json")
    return main_result_train


## Générer le jeu de donnée

In [ ]:
### FAIRE LES JEUX DE DONNEES ENSEMBLES ( sans décider des indices )
main_result_train, main_result_test = make_train_test(randomize=True, pourcentTrain = 0.7)

In [ ]:
### SEPARER TRAIN ET TEST SI ON A DES INDICES SPECIFIQUES
main_result_test = test_panda(0, 6, xmls_name)
main_result_train = train_panda(7, 15, xmls_name)

In [ ]:
print("Il y a",  len(os.listdir(train_path)), "images de train, et", len(os.listdir(test_path)), "images de test")

In [ ]:
main_result_train

In [ ]:
main_result_test